## AQS Data Exploration and Cleaning

This notebook explores, cleans and prepares the data collected from the API requests from the EPA website for import into Tableau. The data gathered in this notebook is available at: https://www.epa.gov/outdoor-air-quality-data. The resulting dataset is saved as a DataFrame and exported as a CSV file titled _'aqs_category_data.csv'_ and an Excel file titled _'aqs_category_data.xlsx'_.

#### Import the Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import pandasql as ps
from datetime import datetime
from collections import Counter

#### Read in the AQS Raw Dataset and AQI Category Dataset

In [ ]:
aqs_df = pd.read_csv('../data/aqs_data_raw.csv')

In [ ]:
aqi_bp = pd.read_csv('../data/aqi_breakpoints.csv')
aqi_bp.columns = aqi_bp.columns.str.lower().str.replace(' ', '_')
aqi_bp = aqi_bp.loc[aqi_bp.aqi_category != 'NONE']

#### Declare the Static Variables

In [ ]:
# A dictionary with counties as keys and associated cities as values
city_county_dict = {'King': 'Seattle',
                    'San Diego': 'San Diego',
                    'Maricopa': 'Phoenix',
                    'Hennepin': 'Minneapolis',
                    'Denver': 'Denver',
                    'Travis': 'Austin',
                    'Philadelphia': 'Philadelphia',
                    'Davidson': 'Nashville',
                    'Duval': 'Jacksonville'}

#### Rename the Relevant Fields

In [ ]:
aqs_df = aqs_df.rename(columns={'units_of_measure':'measure_units',
                                'first_max_value':'max_value1',
                                'first_max_datetime':'max_date1', 
                                'second_max_value':'max_value2', 
                                'second_max_datetime':'max_date2',
                                'third_max_value':'max_value3', 
                                'third_max_datetime':'max_date3', 
                                'fourth_max_value':'max_value4', 
                                'fourth_max_datetime':'max_date4',
                                'ninety_ninth_percentile':'percentile99',
                                'ninety_fifth_percentile':'percentile95',
                                'ninetieth_percentile':'percentile90',
                                'seventy_fifth_percentile':'percentile75',
                                'sample_duration_code':'duration_code'})

#### Create a Calculated Field for the Average of the Four Max Measurement Values

In [ ]:
aqs_df['avg_max_value'] = [round(np.mean([row.max_value1, row.max_value2, row.max_value3, row.max_value4]), 1) for ind, row in aqs_df.iterrows()]

In [ ]:
aqs_df.parameter.value_counts()

#### Add Associated Cities (for potential merging purposes)

In [ ]:
aqs_df['assoc_city'] = [city_county_dict[row.county] for index, row in aqs_df.iterrows()]

#### Subset for Valid Parameters

In [ ]:
county_counts = pd.DataFrame(aqs_df.groupby(['parameter_code', 'year']).county.nunique())
county_counts = county_counts.rename(columns={'county':'freqency'}).reset_index()

In [ ]:
param_county_cnt = {}
for ind, row in county_counts.iterrows():
    if row.freqency == 9:
        if row.parameter_code in param_county_cnt.keys():
            param_county_cnt[row.parameter_code] += 1
        else:
            param_county_cnt[row.parameter_code] = 1

In [ ]:
valid_params = [param for param in param_county_cnt if param_county_cnt[param] >= 3]
aqs_df = aqs_df.loc[aqs_df.parameter_code.isin(valid_params)]
aqs_df.parameter_code.unique()

In [ ]:
# After exploratory analysis, it was pertinent to only keep the 6 parameters used in 
# the AQI Breakpoints dataset, excluding 88502 which is not used in NAAQS decisions.
# The code below filters for those parameters...
aqi_params = list(aqi_bp.parameter_code.unique())
aqi_params.remove(88502)

aqs_df = aqs_df.loc[aqs_df.parameter_code.isin(aqi_params)]
aqs_df.parameter.unique()

#### Format the Max Value Date Fields to "YYYY-MM-DD"
Optional: Classify each Max Value Date by its Season
Optional Future Step: Calculate the Average Max Value for Each Season

In [ ]:
columns = ['max_date1', 'max_date2', 'max_date3', 'max_date4']

def get_season(value):
    # This function returns the season associated with a date
    # The format of the input should be 'MM-DD'
    if (value >= '03-20') & (value <= '06-20'):
        return 'Spring'
    if (value >= '06-21') & (value <= '09-22'):
        return 'Summer'
    if (value >= '09-23') & (value <= '12-20'):
        return 'Fall'
    if ((value >= '12-21') & (value <= '12-31') | 
        (value >= '01-01') & (value <= '03-19')):
        return 'Winter'
    else:
        return 'NA'
    
for col in columns:
    # Format the date column
    aqs_df[col] = pd.to_datetime(aqs_df[col]).dt.date 
    
    num = col[-1:]
    for ind, row in aqs_df.iterrows():
        # Categorize each date by season
        aqs_df.at[ind, 'season' + num] = get_season(str(row[col])[-5:])

#### Subset for and Reorder Relevant Fields

In [ ]:
aqs_df = aqs_df[['parameter', 'year', 'state', 'assoc_city', 'county', 
                 'max_date1', 'season1', 'max_value1', 
                 'max_date2', 'season2', 'max_value2', 
                 'max_date3', 'season3', 'max_value3', 
                 'max_date4', 'season4', 'max_value4', 
                 'avg_max_value', 'measure_units', 
                 'percentile99', 'percentile95', 'percentile90', 'percentile75', 
                 'parameter_code', 'duration_code', 'sample_duration']]

In [ ]:
measures = pd.DataFrame(aqs_df.groupby(['parameter', 'measure_units']).parameter.count())
measures = measures.rename(columns={'parameter':'frequency'}).reset_index()
measures.sort_values(by='measure_units', ascending=False).reset_index(drop=True)

In [ ]:
aqs_df.to_csv(r'../data/aqs_data_cleaned.csv', index=False)
aqs_df.to_excel(r'../data/aqs_data_cleaned.xlsx', sheet_name='aqs_data_cleaned', index=False)

In [ ]:
aqs_df.shape

### Merge the AQI Category Data with the AQS Parameter Data

In [ ]:
aqi_params = list(aqi_bp.parameter.unique())
aqs_params = list(aqs_df.parameter.unique())
unique_params = [param for param in aqs_params if param not in aqi_params]
print(unique_params)

In [ ]:
query = """
        SELECT aqs_df.year,
               aqs_df.state,
               aqs_df.assoc_city,
               aqs_df.county,
               aqs_df.parameter,
               aqs_df.sample_duration AS duration,
               aqs_df.avg_max_value,
               aqi_bp.low_breakpoint,
               aqi_bp.high_breakpoint,
               aqs_df.measure_units,
               aqi_bp.aqi_category
        FROM aqs_df
        INNER JOIN aqi_bp
        ON (aqs_df.parameter = aqi_bp.parameter) &
           (aqs_df.parameter_code = aqi_bp.parameter_code) &
           (aqs_df.duration_code = aqi_bp.duration_code) &
           (aqs_df.sample_duration = aqi_bp.duration_description) &
           (aqs_df.avg_max_value BETWEEN aqi_bp.low_breakpoint AND aqi_bp.high_breakpoint)
        """

aqs_category_data = ps.sqldf(query,locals())

In [ ]:
aqs_category_data.shape

In [ ]:
aqs_category_data.to_csv(r'../data/aqs_category_data.csv', index=False)
aqs_category_data.to_excel(r'../data/aqs_category_data.xlsx', sheet_name='aqs_category_data', index=False)